# Utilities

This notebook covers utility workflows for `sample_points` and `graph_to_gdf`.

The examples are intentionally concise and focus on:
- generating input point GeoDataFrames with `sample_points`
- converting a generated graph back into node/edge GeoDataFrames with `graph_to_gdf`
- preparing export-ready outputs


In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon

from pysgn import geo_watts_strogatz_network
from pysgn.utils import graph_to_gdf, sample_points


## 1) Generate points with `sample_points`

`sample_points` supports sampling in a bounding box, a polygon, or their intersection.


In [ ]:
# Uniform sampling in a bounding box (projected CRS)
points_bbox = sample_points(
    200,
    bbox=(0, 0, 10_000, 8_000),
    random_state=42,
    crs="EPSG:3857",
)

points_bbox.head()


In [ ]:
# Sampling constrained by a polygon domain
domain = Polygon(
    [
        (0, 0),
        (10_000, 0),
        (10_000, 6_000),
        (6_000, 6_000),
        (6_000, 10_000),
        (0, 10_000),
    ]
)

points_polygon = sample_points(
    200,
    polygon=domain,
    random_state=42,
    crs="EPSG:3857",
)

ax = points_polygon.plot(figsize=(5, 5), markersize=8, alpha=0.8)
gpd.GeoSeries([domain], crs="EPSG:3857").boundary.plot(ax=ax, color="black", linewidth=1)
ax.set_title("sample_points within polygon domain")
ax.set_axis_off()


## 2) End-to-end: points -> graph -> GeoDataFrames

Generate points, build a geospatial graph, then convert it back to node and edge GeoDataFrames using `graph_to_gdf`.


In [ ]:
points = sample_points(
    300,
    bbox=(0, 0, 12_000, 12_000),
    random_state=7,
    crs="EPSG:3857",
)

graph = geo_watts_strogatz_network(points, k=8, p=0.15, random_state=7)

nodes_gdf, edges_gdf = graph_to_gdf(graph)

print(f"Nodes: {len(nodes_gdf):,}, Edges: {len(edges_gdf):,}")
nodes_gdf.head(3)


In [ ]:
edges_gdf.head(3)


## 3) Export-ready outputs

Select stable columns and project to EPSG:4326 for common exchange formats (GeoJSON, geopackage, etc.).


In [ ]:
nodes_export = nodes_gdf[["geometry"]].to_crs("EPSG:4326")
edges_export = edges_gdf[["source", "target", "geometry"]].to_crs("EPSG:4326")

print(nodes_export.crs, edges_export.crs)
print(f"nodes_export: {len(nodes_export):,} rows")
print(f"edges_export: {len(edges_export):,} rows")

# Optional file export
# nodes_export.to_file("nodes.geojson", driver="GeoJSON")
# edges_export.to_file("edges.geojson", driver="GeoJSON")


## See also

- Previous: [Network Models: WS + BA](network_models_ws_ba.ipynb)
- Start here: [Getting Started](getting_started.ipynb)
- Next: [Advanced Options and Performance](advanced_options_performance.ipynb)
